In [ ]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2023-06-21 09:47:29--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15460248 (15M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.74M  54.3MB/s    in 0.3s    

2023-06-21 09:47:29 (54.3 MB/s) - ‘rus-eng.zip’ saved [15460248/15460248]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [ ]:

!tail rus.txt

We need to uphold laws against discrimination — in hiring, and in housing, and in education, and in the criminal justice system. That is what our Constitution and our highest ideals require.	Нам нужно отстаивать законы против дискриминации при найме на работу, в жилищной сфере, в сфере образования и правоохранительной системе. Этого требуют наша Конституция и высшие идеалы.	CC-BY 2.0 (France) Attribution: tatoeba.org #5762728 (BHO) & #6390439 (odexed)
I've heard that you should never date anyone who is less than half your age plus seven. Tom is now 30 years old and Mary is 17. How many years will Tom need to wait until he can start dating Mary?	Я слышал, что никогда не следует встречаться с кем-то вдвое младше вас плюс семь лет. Тому 30 лет, a Мэри 17. Сколько лет Тому нужно ждать до тех пор, пока он сможет начать встречаться с Мэри?	CC-BY 2.0 (France) Attribution: tatoeba.org #10068197 (CK) & #10644473 (notenoughsun)
I do have one final ask of you as your president, the same thing I a

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?А-Яа-яёЁ]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('rus.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:-1]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 467119 sentence pairs
Trimmed to 27228 sentence pairs
Counting words...
Counted words:
rus 9939
eng 4229
['мы все на однои стороне .', 'we re all on the same side .']


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

0m 59s (- 13m 55s) (5000 6%) 3.1163
1m 54s (- 12m 26s) (10000 13%) 2.6301
2m 49s (- 11m 19s) (15000 20%) 2.3558
3m 43s (- 10m 14s) (20000 26%) 2.1441
4m 38s (- 9m 16s) (25000 33%) 1.9697
5m 32s (- 8m 18s) (30000 40%) 1.8148
6m 26s (- 7m 21s) (35000 46%) 1.7260
7m 20s (- 6m 25s) (40000 53%) 1.6222
8m 14s (- 5m 29s) (45000 60%) 1.5344
9m 9s (- 4m 34s) (50000 66%) 1.4875
10m 2s (- 3m 39s) (55000 73%) 1.3979
10m 56s (- 2m 44s) (60000 80%) 1.3655
11m 50s (- 1m 49s) (65000 86%) 1.3080
12m 44s (- 0m 54s) (70000 93%) 1.2327
13m 38s (- 0m 0s) (75000 100%) 1.1861


In [ ]:
evaluateRandomly(encoder1, decoder1)

> я сегодня выходнои .
= i m off today .
< i m a today today . <EOS>

> я жду свою подругу .
= i m waiting for my girlfriend .
< i m waiting for my life . <EOS>

> вы себе противоречите .
= you re contradicting yourselves .
< you re contradicting yourself . <EOS>

> вы у меня в руках .
= you re in my hands .
< you re in my hands . <EOS>

> мы устраиваем тому вечеринку на день рождения .
= we re throwing tom a birthday party .
< we re expecting tom a a party . <EOS>

> их там никогда нет .
= they re never there .
< they re never there . <EOS>

> я не злюсь на тома .
= i m not mad at tom .
< i m not mad at tom . <EOS>

> он убирается в шкафу .
= he s cleaning out his closet .
< he s the out of the . <EOS>

> я рад что ты поправился .
= i m glad you ve recovered .
< i m glad you ve recovered . <EOS>

> я уверен что у него получится .
= i m sure that he ll succeed .
< i m sure that he will succeed . <EOS>



In [ ]:
class EncoderRNN_2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN_2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.gru_2 = nn.GRU(hidden_size, hidden_size)


    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded

        output, hidden = self.gru(output, hidden)
        output, hidden = self.gru_2(output, hidden)

        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN_2(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN_2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.gru_2 = nn.GRU(hidden_size, hidden_size)

        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output, hidden = self.gru_2(output, hidden)

        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN_2(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN_2(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

6m 36s (- 92m 28s) (5000 6%) 3.3091
13m 0s (- 84m 36s) (10000 13%) 2.9153
19m 34s (- 78m 17s) (15000 20%) 2.7432
26m 10s (- 71m 58s) (20000 26%) 2.6181
32m 58s (- 65m 56s) (25000 33%) 2.4974
39m 42s (- 59m 34s) (30000 40%) 2.3459
46m 23s (- 53m 1s) (35000 46%) 2.2431
53m 2s (- 46m 24s) (40000 53%) 2.1287
59m 40s (- 39m 47s) (45000 60%) 2.0306
66m 23s (- 33m 11s) (50000 66%) 1.9391
73m 1s (- 26m 33s) (55000 73%) 1.8661
79m 41s (- 19m 55s) (60000 80%) 1.7960
86m 21s (- 13m 17s) (65000 86%) 1.7235
92m 58s (- 6m 38s) (70000 93%) 1.6648
99m 30s (- 0m 0s) (75000 100%) 1.5898


In [ ]:
evaluateRandomly(encoder1, decoder1)

> они чистые .
= they re clean .
< they re happy . <EOS>

> мы просим вас не делать этого .
= we re asking you not to do that .
< we re asking you not to do that . <EOS>

> я в плохом настроении .
= i m in a bad mood .
< i m in a bad mood . <EOS>

> ты мне не поверишь .
= you re not going to believe me .
< you re not going to . . <EOS>

> он обедает .
= he is having lunch .
< he is a . <EOS>

> вы опять здесь .
= you re back again .
< you re early here . <EOS>

> я глупыи .
= i m stupid .
< i m a . <EOS>

> они студентки .
= they re students .
< they re young . <EOS>

> я все еще в австралии .
= i m still in australia .
< i m still in australia . <EOS>

> я устала так же как и ты .
= i m just as tired as you are .
< i m as strong as you are . <EOS>



In [ ]:
class EncoderRNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN_LSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size,num_layers=1)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded


        output, hidden = self.lstm(output,hidden )
        return output, hidden

    def initHidden(self):
        h_0, c_0 = torch.zeros(2, 1, 1, self.hidden_size, device=device)

        return h_0, c_0

In [ ]:
class DecoderRNN_LSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN_LSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)


        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)


        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN_LSTM(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN_LSTM(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 2s (- 14m 40s) (5000 6%) 3.2141
2m 1s (- 13m 6s) (10000 13%) 2.7587
3m 0s (- 12m 3s) (15000 20%) 2.5622
3m 59s (- 10m 57s) (20000 26%) 2.3531
4m 56s (- 9m 53s) (25000 33%) 2.2118
5m 55s (- 8m 53s) (30000 40%) 2.0824
6m 54s (- 7m 54s) (35000 46%) 1.9858
7m 53s (- 6m 54s) (40000 53%) 1.8880
8m 56s (- 5m 57s) (45000 60%) 1.8127
9m 55s (- 4m 57s) (50000 66%) 1.7303
11m 5s (- 4m 1s) (55000 73%) 1.6780
12m 8s (- 3m 2s) (60000 80%) 1.5777
13m 8s (- 2m 1s) (65000 86%) 1.5299
14m 8s (- 1m 0s) (70000 93%) 1.4794
15m 5s (- 0m 0s) (75000 100%) 1.3910


In [ ]:
evaluateRandomly(encoder1, decoder1)

> я уверен что том едет .
= i m sure tom is on his way .
< i m sure tom will be . <EOS>

> вы такая злая .
= you re so mean .
< you re so funny . <EOS>

> мы не солдаты .
= we re not soldiers .
< we re not a . <EOS>

> я для тебя слишком старыи .
= i m too old for you .
< i m too old for you . <EOS>

> я в ужасе .
= i m horrified .
< i m in . <EOS>

> мы рассчитываем на вашу финансовую помощь .
= we are counting on you for financial help .
< we re counting on you . <EOS>

> вы так молоды .
= you re so young .
< you re so young . <EOS>

> они все еще в доступе .
= they re still available .
< they re still in . <EOS>

> простите что обидел вас .
= i m sorry that i hurt you .
< i m sorry i hurt you . <EOS>

> вы просто констатируете очевидное .
= you are just stating the obvious .
< you re just the wrong . <EOS>



In [ ]:
class EncoderRNN_LSTM_2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN_LSTM_2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size,num_layers=1)
        self.lstm_2 = nn.LSTM(hidden_size, hidden_size,num_layers=1)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded


        output, hidden = self.lstm(output,hidden )
        output, hidden = self.lstm_2(output,hidden )

        return output, hidden

    def initHidden(self):
        h_0, c_0 = torch.zeros(2, 1, 1, self.hidden_size, device=device)

        return h_0, c_0

In [ ]:
class DecoderRNN_LSTM_2(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN_LSTM_2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.lstm_2 = nn.LSTM(hidden_size, hidden_size)


        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output, hidden = self.lstm_2(output, hidden)



        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN_LSTM_2(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN_LSTM_2(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

10m 2s (- 140m 33s) (5000 6%) 3.3517
20m 9s (- 131m 4s) (10000 13%) 3.1232
30m 21s (- 121m 25s) (15000 20%) 3.0408
40m 39s (- 111m 49s) (20000 26%) 2.9779
50m 53s (- 101m 47s) (25000 33%) 2.7702
61m 12s (- 91m 49s) (30000 40%) 2.6848
71m 43s (- 81m 57s) (35000 46%) 2.6350
82m 17s (- 72m 0s) (40000 53%) 2.5830
92m 35s (- 61m 43s) (45000 60%) 2.5534
103m 19s (- 51m 39s) (50000 66%) 2.4694
114m 4s (- 41m 28s) (55000 73%) 2.3959
124m 48s (- 31m 12s) (60000 80%) 2.3742
135m 32s (- 20m 51s) (65000 86%) 2.2904
146m 26s (- 10m 27s) (70000 93%) 2.2644
157m 8s (- 0m 0s) (75000 100%) 2.1784


In [ ]:
evaluateRandomly(encoder1, decoder1)

> вам не угодишь .
= you re a hard person to please .
< you re not as young as i . <EOS>

> ты ведь не учитель ?
= you re not a teacher are you ?
< you re not aren are you you ? <EOS>

> я не собираюсь вам ничего говорить .
= i m not going to tell you anything .
< i m not going to go you . <EOS>

> ты теперь мужчина .
= you re a man now .
< you re a little . <EOS>

> у меня перерыв .
= i m taking a break .
< i m an . <EOS>

> я почти дописал отчет .
= i m almost finished writing the report .
< i m just an adult . <EOS>

> я человек широких взглядов .
= i m open minded .
< i m a little . . <EOS>

> я впервые женюсь .
= i m getting married for the first time .
< i m a . . <EOS>

> ты самыи умныи у нас в классе .
= you re the smartest one in our class .
< you re going to to than i . <EOS>

> я откладываю на старость .
= i m saving up for my old age .
< i m a to . . <EOS>

